Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
from datetime import datetime as dt
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
data_root = '../../input/notMNIST/' # Change me to store data elsewhere
pickle_file = data_root + 'notMNIST.pickle'

print('pickle file location: %s' % pickle_file)

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

pickle file location: ../../input/notMNIST/notMNIST.pickle
Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [6]:
# Define a fully connected layer
def fc_layer(input_x, channels_in, channels_out, layer_name='Full_Connection_Layer'):
    with tf.name_scope(layer_name):
        # It is not a good idea to set initial value as zero
        # It will cause problem during the learning activity
        # w = tf.Variable(tf.zeros([channels_in, channels_out]))

        # These are the parameters that we are going to be training. The weight
        # matrix will be initialized using random values following a (truncated)
        # normal distribution.
        weights = tf.Variable(tf.truncated_normal([channels_in, channels_out]), name='W')

        # The biases get initialized to zero.
        biases = tf.Variable(tf.zeros([channels_out]), name='B')
        fc_conn = tf.matmul(input_x, weights)
        act = tf.nn.relu(fc_conn + biases)

        tf.summary.histogram("weights", weights)
        tf.summary.histogram("biases", biases)
        tf.summary.histogram("fc_conn", fc_conn)

        return act


# Define a Convolutional layer
def conv_layer(input_x, channels_in, channels_out, layer_name='Convolutional_Layer'):
    with tf.name_scope(layer_name):
        weights = tf.Variable(tf.zeros([5, 5, channels_in, channels_out]), name='W')
        biases = tf.Variable(tf.zeros(channels_out), name='B')
        conv_conn = tf.nn.conv2d(input_x, weights, strides=[1, 1, 1, 1], padding='SAME')
        act = tf.nn.relu(conv_conn + biases)

        tf.summary.histogram("weights", weights)
        tf.summary.histogram("biases", biases)
        tf.summary.histogram("conv_conn", conv_conn)

        return act
    

In [8]:
# build the network graph

batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    with tf.name_scope('Input_X'):
        tf_train_dataset = tf.placeholder(tf.float32, shape=(None, image_size * image_size), name='Train_X')
        # tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size), name='Train_X')
        # tf_valid_dataset = tf.constant(valid_dataset, name='Valid_X')
        # tf_test_dataset = tf.constant(test_dataset, name='Test_X')

    with tf.name_scope('Labels_y'):
        # tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels), name='T_y')
        tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels), name='T_y')

    with tf.name_scope('Data_reshape_for_image'):
        # image for display purpose
        tf_train_ds_image = tf.reshape(tf_train_dataset, [-1, 28, 28, 1])
        # tensorboard logging
        tf.summary.image('input', tf_train_ds_image, 3)

    # Create the network
    full_conn_1 = fc_layer(tf_train_dataset, image_size * image_size, num_labels, layer_name='fc_conn_1')
    logits = full_conn_1

    with tf.name_scope('loss_function'):
        with tf.variable_scope('fc_conn_1', reuse=tf.AUTO_REUSE):
            fc_conn_w = tf.get_variable("W", [image_size * image_size, num_labels])

        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels,
                                                                         logits=logits), name='cross_entropy') \
            + tf.nn.l2_loss(fc_conn_w)

        tf.summary.scalar('cross_entropy', loss)
        

    # Optimizer.
    with tf.name_scope('Optimizer'):
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Accuracy
    with tf.name_scope('Accuracy'):
        prediction = logits
        correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(tf_train_labels, 1))
        accuracy_res = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar('accuracy_res', accuracy_res)



In [13]:
# Run the model

num_steps = 3001

with tf.Session(graph=graph) as session:

    # Initialize all the variables
    # session.run(tf.global_variables_initializer())
    tf.global_variables_initializer().run()
    print("Initialized")

    # Make the tensorboard log writer
    session_log_dir = "logs/3_2/" + dt.today().strftime('%Y%m%d%H%M%S')
    writer = tf.summary.FileWriter(session_log_dir)
    print("Logging Directory : %s" % session_log_dir)
    
    writer.add_graph(session.graph)

    # Merge all the tf summary
    merged_summary = tf.summary.merge_all()

    # Data Set
    # Minibatch will be built in loop
    valid_feed_dict = {tf_train_dataset: valid_dataset, tf_train_labels: valid_labels}
    test_feed_dict = {tf_train_dataset: test_dataset, tf_train_labels: test_labels}

    for step in range(num_steps):

        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        train_feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}

        if step % 5 == 0:
            s = session.run(merged_summary, feed_dict=train_feed_dict)
            writer.add_summary(s, step)

        _, l, train_prediction = session.run([optimizer, loss, prediction], feed_dict=train_feed_dict)

        if step % 500 == 0:
            # Predictions for the validation, and test data.

            valid_prediction = session.run(logits, feed_dict=valid_feed_dict)
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(train_prediction, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction, valid_labels))

    test_prediction = session.run(logits, feed_dict=test_feed_dict)
    print("Test accuracy: %.1f%%" % accuracy(test_prediction, test_labels))


Initialized
Logging Directory : logs/3_2/20180814175632
Minibatch loss at step 0: 23.973728
Minibatch accuracy: 8.6%
Validation accuracy: 13.1%
Minibatch loss at step 500: 1.101150
Minibatch accuracy: 73.4%
Validation accuracy: 72.6%
Minibatch loss at step 1000: 1.108867
Minibatch accuracy: 73.4%
Validation accuracy: 74.8%
Minibatch loss at step 1500: 0.806843
Minibatch accuracy: 81.2%
Validation accuracy: 75.6%
Minibatch loss at step 2000: 0.741570
Minibatch accuracy: 82.8%
Validation accuracy: 76.7%
Minibatch loss at step 2500: 0.705916
Minibatch accuracy: 80.5%
Validation accuracy: 77.5%
Minibatch loss at step 3000: 0.773560
Minibatch accuracy: 76.6%
Validation accuracy: 78.2%
Test accuracy: 85.7%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
# Run the model

num_steps = 3001

with tf.Session(graph=graph) as session:

    # Initialize all the variables
    # session.run(tf.global_variables_initializer())
    tf.global_variables_initializer().run()
    print("Initialized")

    # Make the tensorboard log writer
    session_log_dir = "logs/3_2/" + dt.today().strftime('%Y%m%d%H%M%S')
    writer = tf.summary.FileWriter(session_log_dir)
    print("Logging Directory : %s" % session_log_dir)
    
    writer.add_graph(session.graph)

    # Merge all the tf summary
    merged_summary = tf.summary.merge_all()

    # Data Set
    # Minibatch will be built in loop
    valid_feed_dict = {tf_train_dataset: valid_dataset, tf_train_labels: valid_labels}
    test_feed_dict = {tf_train_dataset: test_dataset, tf_train_labels: test_labels}

    for step in range(num_steps):

        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        # offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        offset = 0

        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        train_feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}

        if step % 5 == 0:
            s = session.run(merged_summary, feed_dict=train_feed_dict)
            writer.add_summary(s, step)

        _, l, train_prediction = session.run([optimizer, loss, prediction], feed_dict=train_feed_dict)

        if step % 500 == 0:
            # Predictions for the validation, and test data.

            valid_prediction = session.run(logits, feed_dict=valid_feed_dict)
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(train_prediction, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction, valid_labels))

    test_prediction = session.run(logits, feed_dict=test_feed_dict)
    print("Test accuracy: %.1f%%" % accuracy(test_prediction, test_labels))

Initialized
Logging Directory : logs/3_2/20180815162020
Minibatch loss at step 0: 22.245453
Minibatch accuracy: 14.1%
Validation accuracy: 10.2%
Minibatch loss at step 500: 0.866742
Minibatch accuracy: 63.3%
Validation accuracy: 30.6%
Minibatch loss at step 1000: 0.865014
Minibatch accuracy: 63.3%
Validation accuracy: 31.8%
Minibatch loss at step 1500: 0.864489
Minibatch accuracy: 63.3%
Validation accuracy: 32.3%
Minibatch loss at step 2000: 0.864230
Minibatch accuracy: 63.3%
Validation accuracy: 32.7%
Minibatch loss at step 2500: 0.864077
Minibatch accuracy: 63.3%
Validation accuracy: 33.0%
Minibatch loss at step 3000: 0.863976
Minibatch accuracy: 63.3%
Validation accuracy: 33.3%
Test accuracy: 36.2%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
